腾讯云cos存储桶服务端：

https://github.com/tencentyun/qcloud-cos-sts-sdk/edit/master

当然，其类似于github图形化管理，也能拖拽文件存储。以下案例展示是本人实习时协助单位将扫描仪里的tiff格式发票文件实行“一键”输入查验，方便介绍腾讯云工作流。

In [ ]:
import cv2
import os.path

# 将扫描仪内tiff格式增值税发票图片转换为png格式图片
img_fold = '/Users/tatiana/Desktop/folder1'
save_fold = '/Users/tatiana/Desktop/folder2'
img_list = os.listdir(img_fold)[2:]
num_imgs = len(img_list)

for i in range(num_imgs):
    pic_name = img_list[i]
    org_path = os.path.join(img_fold, pic_name)
    img = cv2.imread(org_path, 1)
    file_name_temp = pic_name[:-4]
    file_name = os.path.join(save_fold , file_name_temp+'.png')
    cv2.imwrite(file_name, img)

In [16]:
import re, json, docx
import pandas as pd
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException 
from tencentcloud.ocr.v20181119 import ocr_client, models 

# 预先将转换后的png图片填入腾讯云cos存储桶
def VatInvoiceOCR(img_path):
    cred = credential.Credential("AKIDZw5zTj9fKlwrX1KJDuMYVCesqhIf6fLn", "NUTDixxiHdjhPs52kETSJuGMCoCLF0Wo") 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "ocr.ap-shanghai.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = ocr_client.OcrClient(cred, "ap-shanghai", clientProfile) 

    req = models.VatInvoiceOCRRequest()
    params = '{"ImageUrl":%s}'%img_path
    req.from_json_string(params)

    resp = client.VatInvoiceOCR(req) 
    return json.loads(resp.to_json_string())

def get_vatInfo(img_path): 
    VAT_temp = {}
    resp = VatInvoiceOCR(img_path)
    for item_dict in resp['VatInvoiceInfos']:
        if item_dict['Name'] == '发票代码':
            VAT_temp['发票代码'] = item_dict['Value']
        elif item_dict['Name'] == '发票号码':
            VAT_temp['发票号码'] = item_dict['Value'][2:]
        elif item_dict['Name'] == '开票日期':
            VAT_temp['开票日期'] = ''.join([x for x in item_dict['Value'] if x.isdigit()])
        elif item_dict['Name'] == '合计金额': 
            VAT_temp['税前金额'] = item_dict['Value'][1:]
    return VAT_temp

def reorganization_pictures():
    file = docx.Document()
    for i in range(picnum):
        # 截屏存储路径,请自行替换
        save_path = "/Users/tatiana/Desktop/VAT" + str(i+1)+".png" 
        file.add_picture(img_path)
        # 整理后的截屏文件存放路径,请自行替换
        file.save('/Users/tatiana/Desktop/VATs_inspection_results.docx') 

if __name__=="__main__":
    
    VATs_table = pd.DataFrame()
    
    picnum = int(input("请输入要查验的增值税发票数量:"))
    for i in range(picnum):   
        img_path = "\"https://vat-1259341270.cos.ap-shanghai.myqcloud.com/VAT/VAT" + str(i+1)+".png\""
        VAT_temp = get_vatInfo(img_path)
        VATs_table = VATs_table.append(VAT_temp, ignore_index=True)
        
    order = ['发票代码','发票号码','开票日期','税前金额']
    VATs_table = VATs_table[order]

请输入要查验的增值税发票数量:2 


In [17]:
VATs_table

,发票代码,发票号码,开票日期,税前金额
0,3300193130,01854505,20191212,93103.45
1,2300163130,00313195,20180501,1002.16
